In [43]:
import pandas as pd
import numpy as np 
import json
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

In [4]:
from google.colab import files
uploaded = files.upload()

Saving train.jsonl to train.jsonl
Saving val_wtihout.jsonl to val_wtihout.jsonl


## Обработка

In [5]:
def transform(data_json_file):
    X, y = [], []
    with open(data_json_file, 'r') as json_file:
        json_list = list(json_file)
        for json_str in json_list:
            item = json.loads(json_str)
            text = item['passage']['text']
            questions = item['passage']['questions']
            for q in questions:
                query = q['question']
                ans = q['answers']
                for a in ans:
                    X.append(text + ' Query: '+ query +' Answer: ' + a['text'])
                    y.append(a['label'])
    return X, y

In [15]:
X_train, y_train = transform('train.jsonl')

train = pd.DataFrame({
    'text': X_train,
    'label': y_train
})

print(train.head())

                                                text  label
0  (1) Но люди не могут существовать без природы,...      1
1  (1) Но люди не могут существовать без природы,...      0
2  (1) Но люди не могут существовать без природы,...      0
3  (1) Но люди не могут существовать без природы,...      1
4  (1) Но люди не могут существовать без природы,...      0


Словоформы

In [ ]:
!pip install pymorphy2
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

In [ ]:
new_text=[]
for i in range(len(train['text'])):
    a = train.loc[i]['text'].split(' ')
    new_words=[]
    for b in a:
        new = morph.parse(b)
        new_words.append(new[0].normal_form)
    text = " ".join(new_words)
    new_text.append(text)

train['text'] = new_text
train 

Знаки

In [17]:
punctuation_signs = list(":!.,:;)(")
for punct_sign in punctuation_signs:
    train['text'] = train['text'].str.replace(punct_sign, '')

In [18]:
train['text'] = train['text'].str.lower()
train['text'] = train['text'].str.replace('"', '')
train['text'] = train['text'].str.replace("\\n", " ") 

стопслова


In [20]:
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
#from string import punctuation
russian_stopwords = stopwords.words("russian")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [23]:
def remove_punct(text):
    table = {33: ' ', 34: ' ', 35: ' ', 36: ' ', 37: ' ', 38: ' ', 39: ' ', 40: ' ',
             41: ' ', 42: ' ', 43: ' ', 44: ' ', 45: ' ', 46: ' ', 47: ' ', 58: ' ',
             59: ' ', 60: ' ', 61: ' ', 62: ' ', 63: ' ', 64: ' ', 91: ' ', 92: ' ',
             93: ' ', 94: ' ', 95: ' ', 96: ' ', 123: ' ', 124: ' ', 125: ' ', 126: ' '}
    return text.translate(table)

train['text'] = train['text'].map(lambda x: remove_punct(x))

Тестовый


In [ ]:
def get_X_test(data_json_file):
    X1 = []
    X2 = []
    X3 = []
    with open(data_json_file, 'r') as json_file:
        json_list = list(json_file)
        for json_str in json_list:
            item = json.loads(json_str)
            text = item['passage']['text']
            questions = item['passage']['questions']
            for q in questions:
                query = q['question']
                ans = q['answers']
                for a in ans:
                    X1.append(text) #текст
                    X2.append(query) #вопрос
                    X3.append(a['text']) #ответ на вопрос
    return X1, X2, X3

X1, X2, X3 = get_X_test('val_wtihout.jsonl')

test = pd.DataFrame({
    'text': X1,
    'query': X2,
    'answer': X3
})

test

In [ ]:
new_text=[]
for i in range(len(test['text'])):
    a = test.loc[i]['text'].split(' ')
    new_words=[]
    for b in a:
        new = morph.parse(b)
        new_words.append(new[0].normal_form)
    text = " ".join(new_words)
    new_text.append(text)

test['text'] = new_text
test 

In [30]:
new_text=[]
for i in range(len(test['answer'])):
    a = test.loc[i]['answer'].split(' ')
    new_words=[]
    for b in a:
        new = morph.parse(b)
        new_words.append(new[0].normal_form)
    text = " ".join(new_words)
    new_text.append(text)

test['answer'] = new_text
test 

,text,query,answer
0,(1) самый первый «остров» архипелаг возникнуть...,Почему Солженицына перевозили спецконвоем?,так перевозить особо важный заключенных.
1,(1) самый первый «остров» архипелаг возникнуть...,Почему Солженицына перевозили спецконвоем?,"потому, что быть эмигрантом."
2,(1) самый первый «остров» архипелаг возникнуть...,Почему Солженицына перевозили спецконвоем?,"потому, что он сам вырыть себя землянку."
3,(1) самый первый «остров» архипелаг возникнуть...,Почему Солженицына перевозили спецконвоем?,"потому, что он побывать на пересылка красный п..."
4,(1) самый первый «остров» архипелаг возникнуть...,Почему Солженицына перевозили спецконвоем?,"потому, что он быть особо важный заключить и н..."
...,...,...,...
2230,"""(1) канадский порноактёр лука рокко маньотта,...",За что осуждён Рокко Маньотта?,изнасилование.
2231,"""(1) канадский порноактёр лука рокко маньотта,...",За что осуждён Рокко Маньотта?,рокко похитить любовник и присвоить он деньги.
2232,"""(1) канадский порноактёр лука рокко маньотта,...",За что осуждён Рокко Маньотта?,убийство и расчленение тела.
2233,"""(1) канадский порноактёр лука рокко маньотта,...",За что осуждён Рокко Маньотта?,рокко убить любовник и расчленить он тело.


In [39]:
punctuation_signs = list(":!.»«,:;)(")
for punct_sign in punctuation_signs:
    test['text'] = test['text'].str.replace(punct_sign, '')

test['text'] = test['text'].str.lower()
test['text'] = test['text'].str.replace('"', '')
test['text'] = test['text'].str.replace("\\n", " ") 

test['query'] = test['query'].str.lower()
test['query'] = test['query'].str.replace('"', '')
test['query'] = test['query'].str.replace("\\n", " ") 

test['answer'] = test['answer'].str.lower()
test['answer'] = test['answer'].str.replace('"', '')
test['answer'] = test['answer'].str.replace("\\n", " ") 

Векторизация



In [41]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

X_train, X_test, y_train, y_test = train_test_split(train['text'], train['label'], test_size=0.33, random_state=42)

vectorizer = CountVectorizer(ngram_range=(3,6), analyzer='word', max_features = 10000)

X_train_vector = vectorizer.fit_transform(X_train).toarray()
X_test_vector = vectorizer.transform(X_test).toarray()

In [46]:
logr = LogisticRegression(random_state = 42)   
randf = RandomForestClassifier(random_state = 42, n_jobs =-1)
dect = DecisionTreeClassifier(random_state = 42)

models = [logr, randf, dect]

for model in models:
    print(model)
    model.fit(X_train_vector, y_train)
    predictions = model.predict(X_test_vector)
    print(classification_report(y_test, predictions))
    print('=' * 50)


LogisticRegression(random_state=42)


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


              precision    recall  f1-score   support

           0       0.51      0.77      0.62      2127
           1       0.36      0.15      0.21      1817

    accuracy                           0.48      3944
   macro avg       0.44      0.46      0.41      3944
weighted avg       0.44      0.48      0.43      3944

RandomForestClassifier(n_jobs=-1, random_state=42)
              precision    recall  f1-score   support

           0       0.50      0.69      0.58      2127
           1       0.33      0.18      0.23      1817

    accuracy                           0.45      3944
   macro avg       0.41      0.43      0.40      3944
weighted avg       0.42      0.45      0.42      3944

DecisionTreeClassifier(random_state=42)
              precision    recall  f1-score   support

           0       0.51      0.76      0.61      2127
           1       0.32      0.13      0.19      1817

    accuracy                           0.47      3944
   macro avg       0.41      0.45    

In [54]:
test_values = test["text"].tolist()
result = vectorizer.transform(test_values)
logr_predictions = logr.predict(result)

In [58]:
answ = pd.DataFrame(logr_predictions, columns=['label'])
answ.reset_index(inplace=True)
answ.shape


(2235, 2)

In [59]:
answ.to_csv('Baklanov_lang_understandig.csv', index=False)
files.download("Baklanov_lang_understandig.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>